In [ ]:
import numpy as np
import torch
import torchvision
from typing import Tuple

In [103]:
from torchvision.datasets import MNIST
from torchvision.transforms import Lambda, ToTensor

def calculate_mean_and_std() -> Tuple[float, float]:
    mean = torch.mean(train_data.data, dtype=torch.float)/255
    std = torch.std(train_data.data.type(torch.FloatTensor))/255
    return mean, std
mean, std = calculate_mean_and_std()

train_data = MNIST(
    root='.',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose([
        ToTensor(),
        Lambda(lambda t : torch.reshape(t, (-1,)))
    ])
)

test_data = MNIST(
    root='.',
    download=True,
    train=False,
    transform=torchvision.transforms.Compose([
        ToTensor(),
        Lambda(lambda t : torch.reshape(t, (-1,)))
    ])
)
# print(train_data.data[0])

In [104]:
mean, std = calculate_mean_and_std()
assert np.isclose(mean, 0.1306, atol=1e-4)
assert np.isclose(std, 0.3081, atol=1e-4)

In [105]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10)

x, y = next(iter(train_loader))
assert len(x.shape) == 2
assert x.shape == (10, 784)